# LLM tokenizer

A tokenizer can be seen as a mapping of words to numbers. Well, not exactly words, but sometimes words, sometimes fragments of words.

In [33]:
%pip install regex tiktoken

Note: you may need to restart the kernel to use updated packages.


In [34]:
import urllib.request
from pathlib import Path

if not Path("tokenizer.json").exists():
    with open("HF_TOKEN") as f:
        HF_TOKEN = f.read()


    display("Downloading llama 3.2 tokenizer")

    opener = urllib.request.build_opener()
    opener.addheaders = [("Authorization", f"Bearer {HF_TOKEN}")]
    urllib.request.install_opener(opener)

    tokenizer_json_path, headers = urllib.request.urlretrieve('https://huggingface.co/meta-llama/Llama-3.2-1B/raw/main/tokenizer.json', "tokenizer.json")
else:
    tokenizer_json_path = "tokenizer.json"

Load the tokenizer data from the JSON file.

The split regex will be used to split the text into tokens

the vocabulary maps the token strings to their corresponding ids

In [35]:
import json
with open(tokenizer_json_path) as f:
    tokenizer_data = json.load(f)
split = next(filter(lambda t: t["type"] == "Split", tokenizer_data["pre_tokenizer"]["pretokenizers"]))
split_regex = split["pattern"]["Regex"]
vocab = {k.replace("Ġ", " ").encode("utf-8"): v for k, v in tokenizer_data["model"]["vocab"].items()}

display(f"Splitting regex: {split_regex}")
display(f"Vocabulary length: {len(vocab)}")

"Splitting regex: (?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\\r\\n\\p{L}\\p{N}]?\\p{L}+|\\p{N}{1,3}| ?[^\\s\\p{L}\\p{N}]+[\\r\\n]*|\\s*[\\r\\n]+|\\s+(?!\\S)|\\s+"

'Vocabulary length: 128000'

Let's try to split a text with this regex

In [36]:
vocab_inv = {v: k for k, v in vocab.items()}

In [37]:
import regex

tok_regex = regex.compile(split_regex)

In [66]:
text = "I'm pretty much fucked. That's my considered opinion. Fucked. Six days into what should be one of the greatest two months of my life, and it's turned into a nightmare."
tokens = [bytes(t, "utf-8") for t in tok_regex.findall(text)]
tokens

[b'I',
 b"'m",
 b' pretty',
 b' much',
 b' fucked',
 b'.',
 b' That',
 b"'s",
 b' my',
 b' considered',
 b' opinion',
 b'.',
 b' Fucked',
 b'.',
 b' Six',
 b' days',
 b' into',
 b' what',
 b' should',
 b' be',
 b' one',
 b' of',
 b' the',
 b' greatest',
 b' two',
 b' months',
 b' of',
 b' my',
 b' life',
 b',',
 b' and',
 b' it',
 b"'s",
 b' turned',
 b' into',
 b' a',
 b' nightmare',
 b'.']

In [67]:
def bpe_encode(
    mergeable_ranks: dict[bytes, int], input: str, visualise: str | None = "colour"
) -> list[int]:
    parts = [bytes([b]) for b in input]
    while True:
        # See the intermediate merges play out!
        # if visualise:
        #     if visualise in ["colour", "color"]:
        #         visualise_tokens(parts)
        #     elif visualise == "simple":
        #         print(parts)

        # Iterate over all pairs and find the pair we want to merge the most
        min_idx = None
        min_rank = None
        for i, pair in enumerate(zip(parts[:-1], parts[1:])):
            rank = mergeable_ranks.get(pair[0] + pair[1])
            if rank is not None and (min_rank is None or rank < min_rank):
                min_idx = i
                min_rank = rank

        # If there were no pairs we could merge, we're done!
        if min_rank is None:
            break
        assert min_idx is not None

        # Otherwise, merge that pair and leave the rest unchanged. Then repeat.
        parts = parts[:min_idx] + [parts[min_idx] + parts[min_idx + 1]] + parts[min_idx + 2 :]

    if visualise:
        print()

    tokens = [mergeable_ranks[part] for part in parts]
    return tokens

In [68]:
bpe_encode(vocab, b"hello", None)

[15339]

In [69]:
vocab_inv[15339]

b'hello'

In [70]:
encoded = []
for token in tokens:
    enc = bpe_encode(vocab, token, None)
    print(enc)
    encoded.extend(enc)

display(encoded)

[40]
[2846]
[5128]
[1790]
[28252]
[13]
[3011]
[596]
[856]
[6646]
[9647]
[13]
[64662]
[13]
[19198]
[2919]
[1139]
[1148]
[1288]
[387]
[832]
[315]
[279]
[12474]
[1403]
[4038]
[315]
[856]
[2324]
[11]
[323]
[433]
[596]
[6656]
[1139]
[264]
[38911]
[13]


[40,
 2846,
 5128,
 1790,
 28252,
 13,
 3011,
 596,
 856,
 6646,
 9647,
 13,
 64662,
 13,
 19198,
 2919,
 1139,
 1148,
 1288,
 387,
 832,
 315,
 279,
 12474,
 1403,
 4038,
 315,
 856,
 2324,
 11,
 323,
 433,
 596,
 6656,
 1139,
 264,
 38911,
 13]

In [71]:
decoded = b""

for t in encoded:
    decoded += vocab_inv[t]
decoded

b"I'm pretty much fucked. That's my considered opinion. Fucked. Six days into what should be one of the greatest two months of my life, and it's turned into a nightmare."

In [72]:
import tiktoken
cl100k_base = tiktoken.get_encoding("cl100k_base")

In [78]:

encoded = []
for token in tok_regex.findall(text):
    enc = bpe_encode(cl100k_base._mergeable_ranks, bytes(token, "utf-8"), None)
    print(enc)
    encoded.extend(enc)

display(encoded)

[40]
[2846]
[5128]
[1790]
[28252]
[13]
[3011]
[596]
[856]
[6646]
[9647]
[13]
[64662]
[13]
[19198]
[2919]
[1139]
[1148]
[1288]
[387]
[832]
[315]
[279]
[12474]
[1403]
[4038]
[315]
[856]
[2324]
[11]
[323]
[433]
[596]
[6656]
[1139]
[264]
[38911]
[13]


[40,
 2846,
 5128,
 1790,
 28252,
 13,
 3011,
 596,
 856,
 6646,
 9647,
 13,
 64662,
 13,
 19198,
 2919,
 1139,
 1148,
 1288,
 387,
 832,
 315,
 279,
 12474,
 1403,
 4038,
 315,
 856,
 2324,
 11,
 323,
 433,
 596,
 6656,
 1139,
 264,
 38911,
 13]

In [79]:
cl100k_base._pat_str

"'(?i:[sdmt]|ll|ve|re)|[^\\r\\n\\p{L}\\p{N}]?+\\p{L}++|\\p{N}{1,3}+| ?[^\\s\\p{L}\\p{N}]++[\\r\\n]*+|\\s++$|\\s*[\\r\\n]|\\s+(?!\\S)|\\s"

In [80]:
split_regex

"(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\\r\\n\\p{L}\\p{N}]?\\p{L}+|\\p{N}{1,3}| ?[^\\s\\p{L}\\p{N}]+[\\r\\n]*|\\s*[\\r\\n]+|\\s+(?!\\S)|\\s+"

In [81]:
regex.compile(cl100k_base._pat_str).findall(text)

['I',
 "'m",
 ' pretty',
 ' much',
 ' fucked',
 '.',
 ' That',
 "'s",
 ' my',
 ' considered',
 ' opinion',
 '.',
 ' Fucked',
 '.',
 ' Six',
 ' days',
 ' into',
 ' what',
 ' should',
 ' be',
 ' one',
 ' of',
 ' the',
 ' greatest',
 ' two',
 ' months',
 ' of',
 ' my',
 ' life',
 ',',
 ' and',
 ' it',
 "'s",
 ' turned',
 ' into',
 ' a',
 ' nightmare',
 '.']

In [84]:
import tiktoken._educational as tike

In [85]:
tke = tike.SimpleBytePairEncoding(pat_str=cl100k_base._pat_str, mergeable_ranks=cl100k_base._mergeable_ranks)

In [86]:
tke.encode(text)

I

'm
'm

 pretty
 pretty
 pretty
 pretty
 pretty
 pretty
 pretty

 much
 much
 much
 much
 much

 fucked
 fucked
 fucked
 fucked
 fucked
 fucked
 fucked

.

 That
 That
 That
 That
 That

's
's

 my
 my
 my

 considered
 considered
 considered
 considered
 considered
 considered
 considered
 considered
 considered
 considered
 considered

 opinion
 opinion
 opinion
 opinion
 opinion
 opinion
 opinion
 opinion

.

 Fucked
 Fucked
 Fucked
 Fucked
 Fucked
 Fucked
 Fucked

.

 Six
 Six
 Six
 Six

 days
 days
 days
 days
 days

 into
 into
 into
 into
 into

 what
 what
 what
 what
 what

 should
 should
 should
 should
 should
 should
 should

 be
 be
 be

 one
 one
 one
 one

 of
 of
 of

 the
 the
 the
 the

 greatest
 greatest
 greatest
 greatest
 greatest
 greatest
 greatest
 greatest
 greatest

 two
 two
 two
 two

 months
 months
 months
 months
 months
 months
 months

 of
 of
 of

 my
 my
 my

 life
 life
 life
 life
 life

,

 and
 and
 and
 and

 it
 it
 it

's
's

 turned
 turn

[40,
 2846,
 5128,
 1790,
 28252,
 13,
 3011,
 596,
 856,
 6646,
 9647,
 13,
 64662,
 13,
 19198,
 2919,
 1139,
 1148,
 1288,
 387,
 832,
 315,
 279,
 12474,
 1403,
 4038,
 315,
 856,
 2324,
 11,
 323,
 433,
 596,
 6656,
 1139,
 264,
 38911,
 13]

In [87]:
encoded = []
for token in tok_regex.findall(text):
    enc = tike.bpe_encode(cl100k_base._mergeable_ranks, bytes(token, "utf-8"), None)
    encoded.extend(enc)

display(encoded)

[40,
 2846,
 5128,
 1790,
 28252,
 13,
 3011,
 596,
 856,
 6646,
 9647,
 13,
 64662,
 13,
 19198,
 2919,
 1139,
 1148,
 1288,
 387,
 832,
 315,
 279,
 12474,
 1403,
 4038,
 315,
 856,
 2324,
 11,
 323,
 433,
 596,
 6656,
 1139,
 264,
 38911,
 13]

In [88]:
tke.decode(encoded)

"I'm pretty much fucked. That's my considered opinion. Fucked. Six days into what should be one of the greatest two months of my life, and it's turned into a nightmare."

In [97]:
t2  =tike.SimpleBytePairEncoding(pat_str=split_regex, mergeable_ranks=vocab)

In [98]:
enc2  =t2.encode(text)

I

'm
'm

 pretty
 pretty
 pretty
 pretty
 pretty
 pretty
 pretty

 much
 much
 much
 much
 much

 fucked
 fucked
 fucked
 fucked
 fucked
 fucked
 fucked

.

 That
 That
 That
 That
 That

's
's

 my
 my
 my

 considered
 considered
 considered
 considered
 considered
 considered
 considered
 considered
 considered
 considered
 considered

 opinion
 opinion
 opinion
 opinion
 opinion
 opinion
 opinion
 opinion

.

 Fucked
 Fucked
 Fucked
 Fucked
 Fucked
 Fucked
 Fucked

.

 Six
 Six
 Six
 Six

 days
 days
 days
 days
 days

 into
 into
 into
 into
 into

 what
 what
 what
 what
 what

 should
 should
 should
 should
 should
 should
 should

 be
 be
 be

 one
 one
 one
 one

 of
 of
 of

 the
 the
 the
 the

 greatest
 greatest
 greatest
 greatest
 greatest
 greatest
 greatest
 greatest
 greatest

 two
 two
 two
 two

 months
 months
 months
 months
 months
 months
 months

 of
 of
 of

 my
 my
 my

 life
 life
 life
 life
 life

,

 and
 and
 and
 and

 it
 it
 it

's
's

 turned
 turn